### Python and Pandas Version

In [1]:
!python --version
!pip freeze | grep pandas

Python 3.11.0
pandas==1.5.1


### Libraries

In [2]:
import pandas as pd
import json

pd.set_option('display.max_columns', None)

PATH_DATA = './bank_enriched_addresses.csv'
PATH_GEOGRAPHICAL_FEATURE = 'geographical_feature.json'

## 1. Load Data

In [3]:
df = pd.read_csv(PATH_DATA)
df.head()

,age,name,address,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,giovanni kessler,"17864 Grace Field\nLonzoborough, IN 28057",unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,domenico swift,"3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948",services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,talon gulgowski,"056 pfannerstill cape st.\neast marta, nj 21058",management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,cayla pagac,"146 gay fort suite 622\nwest kathryn, la 81781",management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,sim ratke,"779 Hegmann Harbors St.\nIsaacmouth, MS 13269-...",blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## 2. Transformations

#### 2.1. Remove all rows where pdays is -1

In [4]:
df = df[df['pdays'].ne(-1)]

#### 2.2. Split name into first name and second name columns (drop name)

In [5]:
df[['first_name', 'second_name']]= df['name'].str.split(' ', expand=True)
df.drop(['name'], axis=1, inplace=True)

#### 2.3. Replace the values in the age column with bucketed values, such that < 10 becomes 0, 10 <= x < 20 becomes 1, etc.

In [6]:
df['age_group'] = df['age'].floordiv(10)

#### 2.4. Replace yes/no values with booleans

In [7]:
def unique_values(s: pd.Series):
    "Takes a Series (s) and return a sorted list of its unique values"
    return sorted(s.astype(str).str.lower().unique().tolist())


def find_columns_with_values(df: pd.DataFrame, values: list):
    """
    Takes a DataFrame (df) and a list of values (values) and return the names of
    those columns whose unique values are the same as the values in values
    """
    cols_len = list(df.columns[ df.nunique() == len(values)])
    vals_low = sorted(map(lambda i: i.lower(), values)) 
    return list(filter(lambda i: unique_values(df[i]) == vals_low, cols_len))
 

# Find those yes-no columns and create the boolean one
yes_no_columns = find_columns_with_values(df, ['yes', 'no'])
if yes_no_columns:
    df[yes_no_columns] = (df[yes_no_columns] == 'yes').astype(bool)

#### 2.5. Replace day and month with a single date column, of the form dd/MM

In [8]:
MONTHS =['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

def clean_number(s: pd.Series):
    return s.astype(str).str.rjust(2, '0')

def months_number(s: pd.Series, values: list):
    """
    Takes a series (s) and a list of values (values) and returns a 
    series in which each value is mapped with the index of that 
    value in values list
    """
    values_low = list(map(lambda i: i.lower(), values))
    values_dict = {v: i+1 for i, v in enumerate(values_low)}
    return s.astype(str).str.lower().map(values_dict)
    

df['date'] = clean_number(df['day']) + '/' + clean_number(months_number(df['month'],  MONTHS))

#### 2.6. Rename the y column “outcome"

In [9]:
df.rename(columns={'y':'outcome'}, inplace=True)

### Let's take a look of the DataFrame

In [10]:
df.head()

,age,address,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,outcome,first_name,second_name,age_group,date
1,33,"3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948",services,married,secondary,False,4789,True,True,cellular,11,may,220,1,339,4,failure,False,domenico,swift,3,11/05
2,35,"056 pfannerstill cape st.\neast marta, nj 21058",management,single,tertiary,False,1350,True,False,cellular,16,apr,185,1,330,1,failure,False,talon,gulgowski,3,16/04
5,35,"3547 Treutel Crossing Apt. 567\nReillytown, AS...",management,single,tertiary,False,747,False,False,cellular,23,feb,141,2,176,3,failure,False,richmond,effertz,3,23/02
6,36,"177CorneliusCampApt.478\nNikolausbury,AA16457-...",self-employed,married,tertiary,False,307,True,False,cellular,14,may,341,1,330,2,other,False,lauren,mcclure,3,14/05
9,43,"03353CORKERYPRAIRIE\NWESTLONNIE,AR87918-1164",services,married,primary,False,-88,True,True,cellular,17,apr,313,1,147,2,failure,False,justina,willms,4,17/04


In [11]:
df.dtypes

age             int64
address        object
job            object
marital        object
education      object
default          bool
balance         int64
housing          bool
loan             bool
contact        object
day             int64
month          object
duration        int64
campaign        int64
pdays           int64
previous        int64
poutcome       object
outcome          bool
first_name     object
second_name    object
age_group       int64
date           object
dtype: object

## 3. Add a column which categorizes geographical features in the address, where present. Note the dirtiness of the address data and that the exact categories :
* “water”, where the address contains e.g. lake, creek
* “relief”, where the address contains e.g. hill, canyon
* “flat”, where the address contains e.g. plain

In [12]:
with open(PATH_GEOGRAPHICAL_FEATURE, 'r') as fp:
    geographical_feature = json.load(fp)

In [13]:
def find_geoagraphica_features(s: pd.Series, features: dict):
    """
    Takes a series (s) and a dictionary (features) with the form
    {'category': ['list', 'of', 'words']} and return a DataFrame
    with the index and the category that belongs that specific row
    """
    
    # Transform the `features` with the shape {'category': ['list', 'of', 'words']}
    # to a dictionary `features_inv` with the shape {'word': 'category'}
    features_inv = dict()
    for group, values in features.items():
        for v in list(values):
            features_inv[v] = group
            
    # serie s to lower
    s = s.astype(str).str.lower()
    # remove digits, new lines, tabs, dots, etc.
    s = s.replace(["\d+", r"\\t|\\n|\\r", "\t|\n|\r", "\."], [" ", " "," ", ""], regex=True)
    # split by space or comma
    s = s.str.strip().str.split(r' |,')
    # remove those words with less than 3 letters
    s = s.map(lambda i: list(filter(lambda j: len(j) >= 3, i)))
    # create a dataframe with a word per row
    aux = pd.DataFrame(s.explode())
    aux.columns = ['word']
    # remove last s if word is in plural
    aux['word'] = aux['word'].astype(str).map(lambda i: i[:-1] if i.endswith('s') else i)
    # create the group and remove rows with no group
    aux['group'] = aux['word'].map(features_inv)
    aux = aux[~aux['group'].isna()].reset_index()
    # in case an index has more than one group takes the first one and rename to geo_category
    aux['geo_category'] = aux.groupby(['index'])['group'].transform(lambda i:list(i)[0])
    # return the index and the geo_category with no duplicates
    return aux[['index', 'geo_category']].drop_duplicates().set_index('index')


In [14]:
df_geo = find_geoagraphica_features(df['address'], geographical_feature)
if 'geo_category' in df:
    df.drop('geo_category', axis=1, inplace=True)
df = df.join(df_geo)
df[~df['geo_category'].isna()].head()

,age,address,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,outcome,first_name,second_name,age_group,date,geo_category
2,35,"056 pfannerstill cape st.\neast marta, nj 21058",management,single,tertiary,False,1350,True,False,cellular,16,apr,185,1,330,1,failure,False,talon,gulgowski,3,16/04,water
14,31,"0036 0'R3illy Gr0v3s Apt. 281\nPark3rvill3, VA...",blue-collar,married,secondary,False,360,True,True,cellular,29,jan,89,1,241,1,failure,False,donavon,witting,3,29/01,flat
70,37,"0170 mohr parks ave.\nconnellytown, ky 29076-5325",management,married,tertiary,False,0,False,False,cellular,16,jul,268,2,182,3,success,True,misael,franecki,3,16/07,flat
76,41,"985 Zelda Bridge Apt. 181\nSipesmouth, PR 14631",blue-collar,divorced,secondary,False,174,True,False,cellular,14,may,367,2,297,1,other,False,roma,friesen,4,14/05,water
79,40,"23563 williams0n plains\nlak3 kal3shir3, ny 52332",unemployed,married,secondary,False,219,True,False,cellular,17,nov,204,2,196,1,failure,False,kole,russel,4,17/11,flat


### The 9.3% of data has geo_category `flat`, the 8.7% `water` and the 7.8% is `relief`

In [15]:
df['geo_category'].value_counts() / len(df) * 100

flat      9.313725
water     8.578431
relief    7.843137
Name: geo_category, dtype: float64

## 4. Group by the feature (if you created it, or by some other field if not) and filter out any empty values, sort by the age bucket (or age if you didn’t do the bucketing), and return a row count.

In [16]:
df_group = pd.DataFrame(df[['geo_category', 'age_group']].value_counts()).reset_index()
df_group.columns = ['geo_category', 'age_group', 'frequency']
df_group.sort_values(['age_group'])

,geo_category,age_group,frequency
8,water,2,10
11,relief,2,6
9,flat,2,9
0,flat,3,37
2,relief,3,21
1,water,3,29
4,flat,4,20
5,water,4,15
3,relief,4,21
6,water,5,12


## 5. Write the row level data from step [2], and aggregated data from step[3] to both CSV and parquet formats.

In [17]:
df = df.merge(df_group, on=['geo_category', 'age_group'])
df.to_csv('./output.csv', index=False)

# dont have pyarrow nor fastparquet in this computer
# df.to_parquet('./output.parquet')

In [18]:
df.head()

,age,address,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,outcome,first_name,second_name,age_group,date,geo_category,frequency
0,35,"056 pfannerstill cape st.\neast marta, nj 21058",management,single,tertiary,False,1350,True,False,cellular,16,apr,185,1,330,1,failure,False,talon,gulgowski,3,16/04,water,29
1,39,"75306 Sterling Isle\nStaceyland, RI 02213-3936",blue-collar,married,secondary,False,0,True,False,cellular,30,apr,53,1,325,1,failure,False,jayne,kertzmann,3,30/04,water,29
2,33,"188 M3dhurst Rapid \nW3st Amarif0rt, M3 97698",management,single,tertiary,False,1240,True,False,cellular,4,feb,505,2,260,1,other,False,britney,hirthe,3,04/02,water,29
3,37,"3623 Barry Bridge \nWinonaborough, GA 64781-0714",management,single,tertiary,False,876,True,False,cellular,15,apr,140,4,327,2,failure,False,fernando,balistreri,3,15/04,water,29
4,32,"95289 Welch River\nKurtisland, HI 57529-8854",management,single,tertiary,False,12607,False,False,cellular,14,aug,203,5,84,1,failure,False,josie,goyette,3,14/08,water,29
